In [1]:
from sklearn import svm,  grid_search
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import cv2
from skimage.feature import hog
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
## read in the data
with_car_files = glob.glob('train_data/vehicles/GTI_Far/*.png') + glob.glob('train_data/vehicles/GTI_Left/*.png') + \
                glob.glob('train_data/vehicles/GTI_MiddleClose/*.png') + glob.glob('train_data/vehicles/GTI_Right/*.png') \
                + glob.glob('train_data/vehicles/KITTI_extracted/*.png')
non_car_files = glob.glob('train_data/non-vehicles/Extras/*.png') + glob.glob('train_data/non-vehicles/GTI/*.png')

with_car_pics = []
non_car_pics = []
for file in with_car_files:
    image = mpimg.imread(file)
    with_car_pics.append(image)
for file in non_car_files:
    image = mpimg.imread(file)
    non_car_pics.append(image)
    
print('Together %d images with car and %d images without car' %(len(with_car_pics), len(non_car_pics)))

Together 8792 images with car and 8968 images without car


In [3]:
## define a function to find hog features of the images
## set vis as true to show the result.

def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False,
                     feature_vec=True):
    return_list = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  block_norm= 'L2-Hys', transform_sqrt=False, 
                                  visualise= vis, feature_vector= feature_vec)
    
    # name returns explicitly
    hog_features = return_list[0]
    if vis:
        hog_image = return_list[1]
        return hog_features, hog_image
    else:
        return hog_features

In [4]:
## combine the features in color space

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [6]:
## generate the labels for datas
Y = []
for i in with_car_pics:
    Y.append(1)
for i in non_car_pics:
    Y.append(0)
print('Finish!')

Finish!


In [7]:
## extra the hog features for training and set the corresponding labels
color_space = 'HLS' # can be RGB, HSV, LUV, HLS, YUV, YCrCb
hogchannel = 'ALL' # can be 0, 1, 2 or 'ALL'
orientation = 9
pixpercell = 8
cellperblock = 2

hog_X_with = hog_extract_features(with_car_pics, cspace = color_space, orient = orientation, pix_per_cell=pixpercell, 
                                  cell_per_block = cellperblock, hog_channel = hogchannel)
hog_X_non = hog_extract_features(non_car_pics, cspace = color_space, orient = orientation, pix_per_cell=pixpercell, 
                                  cell_per_block = cellperblock, hog_channel = hogchannel)

print('Finish!')

Finish!


In [8]:
## extra the histogram color features for training and set the corresponding labels
colorspace='RGB'
spatialsize=(32, 32)
histbins=32
histrange=(0, 256)

his_X_with = his_extract_features(with_car_pics, cspace=colorspace, spatial_size=spatialsize, 
                                  hist_bins=histbins, hist_range=histrange)
his_X_non = his_extract_features(non_car_pics, cspace=colorspace, spatial_size=spatialsize, 
                                  hist_bins=histbins, hist_range=histrange)
print('Finish!')

Finish!


In [27]:
## define a function to grap the data together

def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if cspace != 'RGB':
            string = 'cv2.COLOR_RGB2' + cspace
            feature_image = cv2.cvtColor(image, eval(string)) 
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)


def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for image in imgs:
        file_features = []
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            string = 'cv2.COLOR_RGB2' + color_space
            feature_image = cv2.cvtColor(image, eval(string))
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [28]:
## set parameters

color_space = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins

In [29]:
## get the data

hoghis_non = extract_features(with_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=False, 
                        hist_feat=True, hog_feat=True)
hoghis_with = extract_features(non_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=False, 
                        hist_feat=True, hog_feat=True)

hoghisspatial_non = extract_features(with_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=True, 
                        hist_feat=True, hog_feat=True)
hoghisspatial_with = extract_features(non_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=True, 
                        hist_feat=True, hog_feat=True)

hogspatial_non = extract_features(with_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=True, 
                        hist_feat=False, hog_feat=True)
hogspatial_with = extract_features(non_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=True, 
                        hist_feat=False, hog_feat=True)

hog_with = extract_features(with_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=False, 
                        hist_feat=False, hog_feat=True)
hog_non = extract_features(non_car_pics, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=False, 
                        hist_feat=False, hog_feat=True)

In [30]:
## split the data into validation(20%) and training set,also shuffle the data. 
X_hog = np.vstack((hog_with, hog_non)).astype(np.float64)
X_hoghis =  np.vstack((hoghis_with, hoghis_non)).astype(np.float64)
X_hogspatial = np.vstack((hogspatial_with, hogspatial_non)).astype(np.float64)
X_hoghisspatial = np.vstack((hoghisspatial_with, hoghisspatial_non)).astype(np.float64)

rand_state = random.randint(0,99)

X_train_hog, X_test_hog, y_train_hog, y_test_hog = train_test_split(
    X_hog, Y, test_size=0.2, random_state=rand_state)

X_train_hoghis, X_test_hoghis, y_train_hoghis, y_test_hoghis = train_test_split(
    X_hoghis, Y, test_size=0.2, random_state=rand_state)

X_train_hogspatial, X_test_hogspatial, y_train_hogspatial, y_test_hogstatial = train_test_split(
    X_hogspatial, Y, test_size=0.2, random_state=rand_state)

X_train_hoghisspatial, X_test_hoghisspatial, y_train_hoghisspatial, y_test_hoghisspatial = train_test_split(
    X_hoghisspatial, Y, test_size=0.2, random_state=rand_state)

print('Finish!')

Finish!


In [36]:
# Fit a per-column scaler
X_scaler_hog = StandardScaler().fit(X_train_hog)
X_scaler_hoghisspatial = StandardScaler().fit(X_train_hoghisspatial)
X_scaler_hogspatial = StandardScaler().fit(X_train_hogspatial)
X_scaler_hoghis = StandardScaler().fit(X_train_hoghis)
# Apply the scaler
scaled_train_X_hog = X_scaler_hog.transform(X_train_hog)
scaled_train_X_hoghis = X_scaler_hoghis.transform(X_train_hoghis)
scaled_train_X_hogspatial = X_scaler_hogspatial.transform(X_train_hogspatial)
scaled_train_X_hoghisspatial = X_scaler_hoghisspatial.transform(X_train_hoghisspatial)

print('Finish!')

Finish!


In [ ]:
## turn the parameters with hog features
print('start tuning the parameters......')
parameters = {'kernel': [ 'linear', 'poly', 'rbf', 'sigmoid'], 'degree':[16, 24, 32], 'coef0': [-8, 0, 8], 'C': [8, 16, 24], 
            'gamma': [8, 16, 24], 'class_weight': [None, 'balanced']}
tune_step = {'degree':8, 'coef0': 8, 'C': 8, 'gamma': 8}
svr = svm.SVC()
old_score = 0.
j = 1

while True:
    print('epoch', j, ', start at:', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    j = j + 1
    clf = grid_search.GridSearchCV(svr, parameters, cv=4, n_jobs = 3)
    clf.fit(scaled_train_X_hog, y_train_hog)
    new_score = clf.best_score_
    print('Parameters: ', clf.best_params_)
    print('Score: ', new_score)
    if new_score < 0.09:
        old_score = new_score
        for i in parameters:
            if i == 'kernel':
                if parameters[i] > 1:
                    parameters[i] = [clf.best_params_[i]]
            if i == 'class_weight':
                continue
            if i == 'degree' and tune_step[i] == 1:
                if len(parameters[i]) > 1:
                    parameters[i] = [clf.best_params_[i]]
                continue
            elif i == 'C':
                if clf.best_params_[i] == parameters[i][1]:
                    tune_step[i] = tune_step[i]/2
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                elif clf.best_params_[i] == parameters[i][0]:
                    parameters[i] = [clf.best_params_[i]/2, clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                else:
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            elif clf.best_params_[i] == parameters[i][1]:
                tune_step[i] = tune_step[i]/2
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            else:
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
    else:
        break


print('tuning finished!')

start tuning the parameters......
epoch 1 , start at: 2018-05-17 15:25:12


In [ ]:
## turn the parameters with color histogramm
print('start tuning the parameters......')
parameters = {'kernel': [ 'linear', 'poly', 'rbf', 'sigmoid'], 'degree':[16, 24, 32], 'coef0': [-8, 0, 8], 'C': [8, 16, 24], 
            'gamma': [8, 16, 24], 'class_weight': [None, 'balanced']}
tune_step = {'degree':8, 'coef0': 8, 'C': 8, 'gamma': 8}
svr = svm.SVC()
old_score = 0.
j = 1

while True:
    print('epoch', j, ', start at:', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    j = j + 1
    clf = grid_search.GridSearchCV(svr, parameters, cv=4, n_jobs = 3)
    clf.fit(scaled_train_X_hoghis, y_train_hoghis)
    new_score = clf.best_score_
    print('Parameters: ', clf.best_params_)
    print('Score: ', new_score)
    if new_score >= 0.09:
        old_score = new_score
        for i in parameters:
            if i == 'class_weight':
                continue
            if i == 'degree' and tune_step[i] == 1:
                if len(parameters[i]) > 1:
                    parameters[i] = [clf.best_params_[i]]
                continue
            elif i == 'C':
                if clf.best_params_[i] == parameters[i][1]:
                    tune_step[i] = tune_step[i]/2
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                elif clf.best_params_[i] == parameters[i][0]:
                    parameters[i] = [clf.best_params_[i]/2, clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                else:
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            elif clf.best_params_[i] == parameters[i][1]:
                tune_step[i] = tune_step[i]/2
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            else:
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
    else:
        break


print('tuning finished!')

In [ ]:
## turn the parameters with color histogramm
print('start tuning the parameters......')
parameters = {'kernel': [ 'linear', 'poly', 'rbf', 'sigmoid'], 'degree':[16, 24, 32], 'coef0': [-8, 0, 8], 'C': [8, 16, 24], 
            'gamma': [8, 16, 24], 'class_weight': [None, 'balanced']}
tune_step = {'degree':8, 'coef0': 8, 'C': 8, 'gamma': 8}
svr = svm.SVC()
old_score = 0.
j = 1

while True:
    print('epoch', j, ', start at:', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    j = j + 1
    clf = grid_search.GridSearchCV(svr, parameters, cv=4, n_jobs = 3)
    clf.fit(scaled_train_X_hogspatial, y_train_hogspatial)
    new_score = clf.best_score_
    print('Parameters: ', clf.best_params_)
    print('Score: ', new_score)
    if new_score >= 0.09:
        old_score = new_score
        for i in parameters:
            if i == 'class_weight':
                continue
            if i == 'degree' and tune_step[i] == 1:
                if len(parameters[i]) > 1:
                    parameters[i] = [clf.best_params_[i]]
                continue
            elif i == 'C':
                if clf.best_params_[i] == parameters[i][1]:
                    tune_step[i] = tune_step[i]/2
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                elif clf.best_params_[i] == parameters[i][0]:
                    parameters[i] = [clf.best_params_[i]/2, clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                else:
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            elif clf.best_params_[i] == parameters[i][1]:
                tune_step[i] = tune_step[i]/2
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            else:
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
    else:
        break


print('tuning finished!')

In [ ]:
## turn the parameters with color histogramm
print('start tuning the parameters......')
parameters = {'kernel': [ 'linear', 'poly', 'rbf', 'sigmoid'], 'degree':[16, 24, 32], 'coef0': [-8, 0, 8], 'C': [8, 16, 24], 
            'gamma': [8, 16, 24], 'class_weight': [None, 'balanced']}
tune_step = {'degree':8, 'coef0': 8, 'C': 8, 'gamma': 8}
svr = svm.SVC()
old_score = 0.
j = 1

while True:
    print('epoch', j, ', start at:', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    j = j + 1
    clf = grid_search.GridSearchCV(svr, parameters, cv=4, n_jobs = 3)
    clf.fit(scaled_train_X_hoghisspatial, y_train_hoghisspatial)
    new_score = clf.best_score_
    print('Parameters: ', clf.best_params_)
    print('Score: ', new_score)
    if new_score >= 0.09:
        old_score = new_score
        for i in parameters:
            if i == 'class_weight':
                continue
            if i == 'degree' and tune_step[i] == 1:
                if len(parameters[i]) > 1:
                    parameters[i] = [clf.best_params_[i]]
                continue
            elif i == 'C':
                if clf.best_params_[i] == parameters[i][1]:
                    tune_step[i] = tune_step[i]/2
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                elif clf.best_params_[i] == parameters[i][0]:
                    parameters[i] = [clf.best_params_[i]/2, clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
                else:
                    parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            elif clf.best_params_[i] == parameters[i][1]:
                tune_step[i] = tune_step[i]/2
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
            else:
                parameters[i] = [clf.best_params_[i]-tune_step[i], clf.best_params_[i], clf.best_params_[i]+tune_step[i]]
    else:
        break


print('tuning finished!')